<a href="https://colab.research.google.com/github/KSustaina/EDXCS50/blob/main/minesweeper_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import copy

class Sentence():
    """
    Represents a logical statement about a set of cells in a Minesweeper game.
    A sentence states that `count` of the `cells` are mines.
    """

    def __init__(self, cells, count):
        """
        Initializes a new Sentence object.

        Args:
            cells: A set of (i, j) tuples representing the cells in the sentence.
            count: An integer representing the number of mines among the cells.
        """
        self.cells = set(cells)
        self.count = count

    def __eq__(self, other):
        """
        Checks if two Sentence objects are equal.

        Args:
            other: Another Sentence object.

        Returns:
            True if the sentences have the same cells and count, False otherwise.
        """
        return self.cells == other.cells and self.count == other.count

    def __str__(self):
        """
        Returns a string representation of the Sentence.
        """
        return f"{self.cells} = {self.count}"

    def known_mines(self):
        """
        Returns the set of cells in self.cells that are known to be mines.
        A cell is known to be a mine if the number of cells in the sentence
        is equal to the count of mines.

        Returns:
            A set of (i, j) tuples representing the cells known to be mines.
        """
        # If the number of cells equals the count, all cells must be mines.
        if len(self.cells) == self.count and self.count > 0:
            return self.cells
        else:
            return set()

    def known_safes(self):
        """
        Returns the set of cells in self.cells that are known to be safe.
        A cell is known to be safe if the count of mines is zero.

        Returns:
            A set of (i, j) tuples representing the cells known to be safe.
        """
        # If the count is zero, all cells must be safe.
        if self.count == 0:
            return self.cells
        else:
            return set()

    def mark_mine(self, cell):
        """
        Updates the sentence by removing a cell known to be a mine.

        If cell is in the sentence, remove it and decrement the count.
        If cell is not in the sentence, no action is necessary.

        Args:
            cell: A (i, j) tuple representing the cell known to be a mine.
        """
        if cell in self.cells:
            self.cells.remove(cell)
            self.count -= 1

    def mark_safe(self, cell):
        """
        Updates the sentence by removing a cell known to be safe.

        If cell is in the sentence, remove it. The count remains unchanged
        as a safe cell does not contribute to the mine count.
        If cell is not in the sentence, no action is necessary.

        Args:
            cell: A (i, j) tuple representing the cell known to be safe.
        """
        if cell in self.cells:
            self.cells.remove(cell)
            # The count does not change when a safe cell is removed

class MinesweeperAI():
    """
    AI for playing Minesweeper.
    """

    def __init__(self, height=8, width=8):
        """
        Initializes a new MinesweeperAI object.

        Args:
            height: The height of the board.
            width: The width of the board.
        """
        # Board dimensions
        self.height = height
        self.width = width

        # Keep track of cells as safe or mines
        self.mines = set()
        self.safes = set()

        # Keep track of moves made by the AI
        self.moves_made = set()

        # Keep track of sentences about the board
        self.knowledge = []

    def mark_mine(self, cell):
        """
        Marks a cell as a mine and updates all knowledge.
        """
        self.mines.add(cell)
        for sentence in self.knowledge:
            sentence.mark_mine(cell)

    def mark_safe(self, cell):
        """
        Marks a cell as safe and updates all knowledge.
        """
        self.safes.add(cell)
        for sentence in self.knowledge:
            sentence.mark_safe(cell)

    def add_knowledge(self, cell, count):
        """
        Called when the Minesweeper board tells us, for a given
        safe cell, how many neighboring cells are mines.

        This function should:
            1) mark the cell as one of the moves made in the game
            2) mark the cell as a safe cell
            3) add a new sentence to the AI's knowledge base based on the
               value of `cell` and `count`
            4) mark any additional cells as safe or as mines if they can be
               inferred from the knowledge base
            5) add any new sentences to the knowledge base if they can be
               inferred from existing sentences
        """
        # 1) Mark the cell as one of the moves made
        self.moves_made.add(cell)

        # 2) Mark the cell as a safe cell
        self.mark_safe(cell)

        # 3) Add a new sentence to the knowledge base
        # Identify neighbors of the cell
        neighbors = set()
        for i in range(cell[0] - 1, cell[0] + 2):
            for j in range(cell[1] - 1, cell[1] + 2):

                # Ignore the cell itself
                if (i, j) == cell:
                    continue

                # Check if cell is on board
                if 0 <= i < self.height and 0 <= j < self.width:
                    neighbors.add((i, j))

        # Create a new sentence from neighbors whose state is unknown
        unknown_neighbors = neighbors - self.mines - self.safes

        # Adjust the count based on known mines among neighbors
        known_mines_in_neighbors = neighbors.intersection(self.mines)
        adjusted_count = count - len(known_mines_in_neighbors)

        # Only add a sentence if there are unknown cells
        if len(unknown_neighbors) > 0:
            new_sentence = Sentence(unknown_neighbors, adjusted_count)
            # Avoid adding duplicate sentences
            if new_sentence not in self.knowledge:
                self.knowledge.append(new_sentence)

        # 4) and 5) Infer new information and sentences
        # This is an iterative process as new inferences might lead to more inferences
        inferences_made = True
        while inferences_made:
            inferences_made = False

            # Check for known mines and safes from existing sentences
            newly_identified_safes = set()
            newly_identified_mines = set()

            for sentence in self.knowledge:
                newly_identified_safes.update(sentence.known_safes())
                newly_identified_mines.update(sentence.known_mines())

            # Mark newly identified safes and mines
            for safe_cell in newly_identified_safes - self.safes:
                self.mark_safe(safe_cell)
                inferences_made = True

            for mine_cell in newly_identified_mines - self.mines:
                self.mark_mine(mine_cell)
                inferences_made = True

            # Remove empty sentences (sentences with no cells left)
            self.knowledge = [s for s in self.knowledge if len(s.cells) > 0]

            # Infer new sentences from existing ones (subset rule)
            newly_inferred_sentences = []
            # Iterate through all pairs of sentences
            for s1 in self.knowledge:
                for s2 in self.knowledge:
                    # Ensure they are different sentences
                    if s1 != s2:
                        # Check if one sentence is a subset of the other
                        if s1.cells.issubset(s2.cells):
                            # If s1 is a subset of s2, then the cells in s2
                            # that are not in s1 must contain the remaining mines.
                            # s2.count - s1.count mines are in s2.cells - s1.cells
                            inferred_cells = s2.cells - s1.cells
                            inferred_count = s2.count - s1.count

                            # Only create a new sentence if there are cells and the count is non-negative
                            if len(inferred_cells) > 0 and inferred_count >= 0:
                                inferred_sentence = Sentence(inferred_cells, inferred_count)
                                # Add the new sentence if it's not already known
                                if inferred_sentence not in self.knowledge and inferred_sentence not in newly_inferred_sentences:
                                    newly_inferred_sentences.append(inferred_sentence)
                                    inferences_made = True # Indicate that a new inference was made

                        elif s2.cells.issubset(s1.cells):
                             # If s2 is a subset of s1, then the cells in s1
                            # that are not in s2 must contain the remaining mines.
                            # s1.count - s2.count mines are in s1.cells - s2.cells
                            inferred_cells = s1.cells - s2.cells
                            inferred_count = s1.count - s2.count

                            # Only create a new sentence if there are cells and the count is non-negative
                            if len(inferred_cells) > 0 and inferred_count >= 0:
                                inferred_sentence = Sentence(inferred_cells, inferred_count)
                                # Add the new sentence if it's not already known
                                if inferred_sentence not in self.knowledge and inferred_sentence not in newly_inferred_sentences:
                                    newly_inferred_sentences.append(inferred_sentence)
                                    inferences_made = True # Indicate that a new inference was made

            # Add all newly inferred sentences to the knowledge base
            self.knowledge.extend(newly_inferred_sentences)

            # After adding new sentences, re-check for known mines/safes
            # from the expanded knowledge base in the next iteration of the while loop


    def make_safe_move(self):
        """
        Returns a safe cell to choose on the Minesweeper board.
        The move must be known to be safe, and not a move already made.

        Returns:
            A (i, j) tuple representing the safe move, or None if no safe move
            can be guaranteed.
        """
        # Iterate through known safes and return the first one not yet made
        for cell in self.safes:
            if cell not in self.moves_made:
                return cell
        return None # No safe move found

    def make_random_move(self):
        """
        Returns a move to make on the Minesweeper board.
        This function may be called if safe moves are not available.
        The move must not be already made and must not be a known mine.

        Returns:
            A (i, j) tuple representing the random move, or None if no possible
            moves remain.
        """
        possible_moves = []
        # Iterate through all cells on the board
        for i in range(self.height):
            for j in range(self.width):
                cell = (i, j)
                # Check if the cell is not a mine and not already made
                if cell not in self.mines and cell not in self.moves_made:
                    possible_moves.append(cell)

        # If there are possible moves, choose one randomly
        if possible_moves:
            return random.choice(possible_moves)
        else:
            return None # No possible moves left